In [76]:
import tensorflow as tf

tf.__version__

'2.19.0'

In [77]:
import tensorflow_datasets as tfds
import os

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [78]:
#load the mnist datasrt

datasets, info  = tfds.load(name="mnist", with_info=True, as_supervised=True)

In [79]:
mnist_train, mnist_test = datasets["train"], datasets["test"]

In [80]:
len(mnist_test), len(mnist_train)

(10000, 60000)

In [81]:
# define the mirred stragy
strategy = tf.distribute.MirroredStrategy()

strategy.num_replicas_in_sync

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


1

In [82]:
num_train_examples = info.splits["train"].num_examples
num_test_examples = info.splits["test"].num_examples

num_test_examples, num_train_examples

(10000, 60000)

In [83]:
BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64

BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

BATCH_SIZE

64

In [84]:
def scale(images, labesl):
    images = tf.cast(images, tf.float32)
    images = images / 255.0
    
    return images, labesl

In [85]:
train_dataset = mnist_train.map(scale).shuffle(BUFFER_SIZE).cache().batch(BATCH_SIZE)
test_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

In [86]:
len(train_dataset), len(test_dataset)

(938, 157)

In [87]:
with strategy.scope():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10)
  ])

  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                metrics=['accuracy'])

/Users/sachinmurali/anaconda3/envs/tensorflows/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [88]:
checkpoint_dir = "./training_checkpoints"
check_point_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

check_point_prefix

'./training_checkpoints/ckpt_{epoch}.weights.h5'

In [89]:
def decay(epoch):
  if epoch < 3:
    return 1e-3
  elif epoch >= 3 and epoch < 7:
    return 1e-4
  else:
    return 1e-5

In [90]:
# Define a callback for printing the learning rate at the end of each epoch.
class PrintLR(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    print('\nLearning rate for epoch {} is {}'.format(        epoch + 1, model.optimizer.learning_rate.value.numpy()))
    


In [91]:
# Put all the callbacks together.
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.ModelCheckpoint(filepath=check_point_prefix,
                                       save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay),
    PrintLR()
]

In [92]:
EPOCHS = 12

model.fit(train_dataset, epochs=EPOCHS, callbacks=callbacks)

W0000 00:00:1749789799.158293  507378 auto_shard.cc:558] The `assert_cardinality` transformation is currently not handled by the auto-shard rewrite and will be removed.
2025-06-13 10:13:19.178314: W tensorflow/core/framework/dataset.cc:993] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


Epoch 1/12


2025-06-13 10:13:19.359670: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-06-13 10:13:19.359800: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


933/938 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8836 - loss: 0.4012
Learning rate for epoch 1 is 0.0010000000474974513
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8839 - loss: 0.3999 - learning_rate: 0.0010
Epoch 2/12
936/938 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9772 - loss: 0.0760
Learning rate for epoch 2 is 0.0010000000474974513
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9772 - loss: 0.0760 - learning_rate: 0.0010
Epoch 3/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9849 - loss: 0.0508
Learning rate for epoch 3 is 0.0010000000474974513
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9849 - loss: 0.0508 - learning_rate: 0.0010
Epoch 4/12
935/938 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9910 - loss: 0.0324
Learning rate for epoch 4 is 9.999999747378752e-05
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9910 - loss: 0.0324 - learning_rate: 1.0000e-04
Epoch 5/12
 11/938 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy:

2025-06-13 10:13:37.906514: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]


931/938 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9928 - loss: 0.0265
Learning rate for epoch 5 is 9.999999747378752e-05
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9928 - loss: 0.0264 - learning_rate: 1.0000e-04
Epoch 6/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9938 - loss: 0.0240
Learning rate for epoch 6 is 9.999999747378752e-05
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9938 - loss: 0.0240 - learning_rate: 1.0000e-04
Epoch 7/12
929/938 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9944 - loss: 0.0219
Learning rate for epoch 7 is 9.999999747378752e-05
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9944 - loss: 0.0219 - learning_rate: 1.0000e-04
Epoch 8/12
 26/938 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9927 - loss: 0.0209

2025-06-13 10:13:50.723326: W tensorflow/core/framework/dataset.cc:993] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


933/938 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9950 - loss: 0.0195
Learning rate for epoch 8 is 9.999999747378752e-06
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9950 - loss: 0.0195 - learning_rate: 1.0000e-05
Epoch 9/12
932/938 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9953 - loss: 0.0190
Learning rate for epoch 9 is 9.999999747378752e-06
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9953 - loss: 0.0189 - learning_rate: 1.0000e-05
Epoch 10/12
935/938 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9954 - loss: 0.0187
Learning rate for epoch 10 is 9.999999747378752e-06
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9954 - loss: 0.0186 - learning_rate: 1.0000e-05
Epoch 11/12
930/938 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9954 - loss: 0.0184
Learning rate for epoch 11 is 9.999999747378752e-06
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9954 - loss: 0.0184 - learning_rate: 1.0000e-05
Epoch 12/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms

2025-06-13 10:14:12.148418: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
